In [1]:
import torch
import torch.nn as nn
from torch.distributions import constraints

import pyro
import pyro.contrib.examples.polyphonic_data_loader as poly
import pyro.distributions as dist
from pyro import poutine
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO, TraceTMC_ELBO
from pyro.infer.autoguide import AutoDelta
from pyro.ops.indexing import Vindex
from pyro.optim import Adam

In [2]:
data = poly.load_data(poly.JSB_CHORALES)

In [3]:
seqs = data['train']['sequences']
lengths = data['train']['sequence_lengths']

In [7]:
hd = 4

In [12]:
batch_size = 10

In [14]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', 'os', 'sys', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'torch', 'nn', 'constraints', 'pyro', 'poly', 'dist', 'poutine', 'SVI', 'JitTraceEnum_ELBO', 'TraceEnum_ELBO', 'TraceTMC_ELBO', 'AutoDelta', 'Vindex', 'Adam', '_i2', 'data', '_i3', 'seqs', 'lengths', '_i4', '_4', '_i5', 'num_seqs', 'max_len', 'data_dim', '_i6', '_i7', 'hd', '_i8', 'probs_x', '_i9', '_9', '_i10', 'probs_y', '_i11', '_11', '_i12', 'batch_size', '_i13', 'tones_plate', '_i14'])

In [1]:
def model_2(sequences, lengths, args, batch_size=None, include_prior=True):
    with ignore_jit_warnings():
        num_sequences, max_length, data_dim = map(int, sequences.shape)
        assert lengths.shape == (num_sequences,)
        assert lengths.max() <= max_length
    with poutine.mask(mask=include_prior):
        probs_x = pyro.sample(
            "probs_x",
            dist.Dirichlet(0.9 * torch.eye(args.hidden_dim) + 0.1).to_event(1),
        )
        probs_y = pyro.sample(
            "probs_y",
            dist.Beta(0.1, 0.9).expand([args.hidden_dim, 2, data_dim]).to_event(3),
        )
    tones_plate = pyro.plate("tones", data_dim, dim=-1)
    with pyro.plate("sequences", num_sequences, batch_size, dim=-2) as batch:
        lengths = lengths[batch]
        x, y = 0, 0
        for t in pyro.markov(range(max_length if args.jit else lengths.max())):
            with poutine.mask(mask=(t < lengths).unsqueeze(-1)):
                x = pyro.sample(
                    "x_{}".format(t),
                    dist.Categorical(probs_x[x]),
                    infer={"enumerate": "parallel"},
                )
                # Note the broadcasting tricks here: to index probs_y on tensors x and y,
                # we also need a final tensor for the tones dimension. This is conveniently
                # provided by the plate associated with that dimension.
                with tones_plate as tones:
                    y = pyro.sample(
                        "y_{}".format(t),
                        dist.Bernoulli(probs_y[x, y, tones]),
                        obs=sequences[batch, t],
                    ).long()